In [2]:
# -------------- #

import pandas as pd
import os
import datetime
import math

import load_data
from util_log import logger
# from featureEng import (
# )
# from models import grid_search
from config import (
    RAW_DATA_DIR,
    LAG_DICT,
    SLIDING_DICT
)
from models.baseline_avg_model import AvgModel
from sklearn.metrics import r2_score, mean_squared_error

import mlflow
import numpy as np
import mlflow.sklearn

# valid period. '2017-07-31' to '2017-08-16'
SPLIT_DATE = datetime.date(2017,7, 31)

def train_valid_split(df):
    train_df = df[df['date']<SPLIT_DATE]
    valid_df = df[df['date']>=SPLIT_DATE]
    return train_df, valid_df

/Users/liuyu/Desktop/g-forecast/config.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


-------------------------------------------------------------
configurations: 
{
  "storeId": null,
  "influence_sheet": "",
  "influence_min": 0,
  "influence_max": 0,
  "predict_start_date": "2019-06-23",
  "predict_length": 35,
  "lag_length": 50,
  "lag_cols": [
    "Sales"
  ],
  "sliding_window_length": 50,
  "sliding_cols": [
    "Sales"
  ],
  "model_list": [
    "LGBMRegressor"
  ],
  "peak_period": [
    "2017-12-08",
    "2018-02-08",
    "2018-12-10",
    "2019-02-10"
  ]
}
-------------------------------------------------------------


In [1]:
def add_weekday(df):
    df['weekday'] = df['date'].dt.weekday
    return df

In [3]:
logger.debug('Get raw data')
train_df, test_df = load_data.main()

# store_list = train_df.store_nbr.unique()[2:7]
# train_df = train_df[train_df['store_nbr'].isin(store_list)]
train_df = train_df.dropna()

train_df = train_df[train_df['date']>=datetime.date(2017, 7, 1)]
logger.debug('Train valid split')
train_df, valid_df = train_valid_split(train_df)

logger.debug('Add weekday')
train_df = add_weekday(train_df)
valid_df = add_weekday(valid_df)

Get raw data
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
Train valid split
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not c

In [4]:
# from config import (
#     SPLIT_DATE
# )

# %%
class AvgModel(object):
    def __init__(
                self, 
                label,
                train_days):
        self.label = label 
        self.train_days = train_days
        return

    def averaging_by_weekday(
                            self, 
                            df_item, 
                            column):
        """
        
        """
        result = {}
        for k in range(7):
            chunk = df_item[df_item['weekday'] == k]
            if self.train_days > 0:
                start_date = chunk['date'].iloc[-1] - \
                    pd.Timedelta(days=self.train_days)
                chunk = chunk[chunk['date'] >= start_date]
                # print(chunk)
            result[k] = chunk[column].mean()
        return result

    def fit(self, df):
        self.model = {}
        self.model_item = {}
        store_list = df['store_nbr'].unique()
        for store_id in store_list:
            df_store = df[df['store_nbr'] == store_id]
            item_list = df_store.item_nbr.unique()
            self.model[store_id] = {}
            for item_id in item_list:
                df_item = df_store[df_store['item_nbr'] == item_id]
                result =  self.averaging_by_weekday(df_item, self.label)
                self.model[store_id][item_id] = result
                if item_id not in self.model_item:
                    pass
                else:
                    o_r = self.model_item[item_id]
                    for k in range(7):
                        result[k] = (result[k]+o_r[k])/2
                self.model_item[item_id] = result
        return

    def predict(self, valid_df):
        def create_fn(row):
            try:
                return self.model[row['store_nbr']][row['item_nbr']][row['weekday']]
            except:
                try:
                    return self.model_item[row['item_nbr']][row['weekday']]
                except:
                    return 0
        pred = valid_df.apply(create_fn, axis=1)
        return pred

In [5]:
avg_model = AvgModel(label='unit_sales',
                    train_days=-1)
avg_model.fit(train_df)
_pred = avg_model.predict(valid_df)
_pred=_pred.fillna(0)
true=valid_df['unit_sales'].values
pred = _pred.values
_score = mean_squared_error(true, pred)

In [7]:
_score

416.22662279519153

In [ ]:
mlflow.set_experiment(model_used)
with mlflow.start_run(run_name=model_used):
    mlflow.log_param('model', model_used)

    if model_used == 'avg':
        avg_model = AvgModel(label='unit_sales',
                            train_days=-1)
        avg_model.fit(train_df)
        _pred = avg_model.predict(valid_df)
        _score = r2_score(valid_df['unit_sales'].values, _pred)
        mlflow.log_param('train_days', 16)

    mlflow.log_metric('r2_score', score)


